# Librerias y Lectura Archivos

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input
from keras_tuner import RandomSearch

In [3]:
#importacion df_final
df=pd.read_csv('../data/processed/v3_final_merge.csv')

In [4]:
df

,edad,sexo,nivel_educativo,fumador,actividad_fisica,vive,diabetes,hipertension,obesidad,cancer,...,region_Sur,tipo_empleo_Desempleado,tipo_empleo_Privado,tipo_empleo_Público,tipo_empleo_Temporal,continente_America,continente_Antarctica,continente_Asia,continente_Europe,continente_Oceania
0,69,0,1,1,0,1,1,1,1,0,...,0,0,0,1,0,0,0,1,0,0
1,32,1,2,0,1,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0
2,89,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,78,1,1,1,0,0,1,0,1,1,...,0,0,0,0,1,0,0,1,0,0
4,38,0,3,1,1,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,21,1,2,0,2,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
49996,35,0,3,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
49997,46,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
49998,56,1,3,1,1,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [5]:
df.shape

(50000, 48)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   edad                     50000 non-null  int64  
 1   sexo                     50000 non-null  int64  
 2   nivel_educativo          50000 non-null  int64  
 3   fumador                  50000 non-null  int64  
 4   actividad_fisica         50000 non-null  int64  
 5   vive                     50000 non-null  int64  
 6   diabetes                 50000 non-null  int64  
 7   hipertension             50000 non-null  int64  
 8   obesidad                 50000 non-null  int64  
 9   cancer                   50000 non-null  int64  
 10  enfermedad_cardiaca      50000 non-null  int64  
 11  epoc                     50000 non-null  int64  
 12  glucosa                  50000 non-null  float64
 13  colesterol               50000 non-null  float64
 14  trigliceridos         

# Estandarización Variables

In [7]:
numerical_columns = ['edad', 'glucosa', 'colesterol', 'trigliceridos', 
                     'hemoglobina', 'leucocitos', 'plaquetas', 'creatinina', 
                     'ingresos_mensuales', 'gastos_salud']

In [8]:
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
df[numerical_columns].head()

,edad,glucosa,colesterol,trigliceridos,hemoglobina,leucocitos,plaquetas,creatinina,ingresos_mensuales,gastos_salud
0,0.750106,1.125931,1.116180,0.518919,-0.262543,-0.924342,0.202956,-0.323535,1.499543,2.487493
1,-1.028843,-1.549388,-1.768373,-1.357231,0.177002,-0.083060,0.709850,-0.883740,2.557298,-0.885562
2,1.711700,-0.436864,-0.699218,-2.059947,-0.742047,0.481094,-0.116266,2.570857,1.082786,-0.885562
3,1.182823,-0.258148,0.287809,-0.946194,-0.552243,1.490633,-0.122279,1.030293,-0.009775,2.487493
4,-0.740365,-1.940861,-1.565293,-2.566843,-1.161613,1.134325,-0.389633,-0.743689,0.011400,-0.885562


## Red Neuronal

In [9]:
X = df.drop(columns='vive')
y = df['vive']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelo Simple

In [11]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # salida binaria
])


In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

In [13]:
# 6. Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step - accuracy: 0.8647 - loss: 0.3802 - precision: 0.8791 - recall: 0.9807 - val_accuracy: 0.8741 - val_loss: 0.3292 - val_precision: 0.8749 - val_recall: 0.9987
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.8752 - loss: 0.3308 - precision: 0.8768 - recall: 0.9973 - val_accuracy: 0.8744 - val_loss: 0.3268 - val_precision: 0.8754 - val_recall: 0.9983
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - accuracy: 0.8763 - loss: 0.3234 - precision: 0.8794 - recall: 0.9953 - val_accuracy: 0.8742 - val_loss: 0.3288 - val_precision: 0.8761 - val_recall: 0.9970
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 465us/step - accuracy: 0.8774 - loss: 0.3220 - precision: 0.8798 - recall: 0.9962 - val_accuracy: 0.8737 - val_loss: 0.3270 - val_precision: 0.8807 - val_recall: 0.9894
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 504us/step - accuracy: 0.8749 - loss: 0.3229 - precision: 0.8795 - recall: 0.9925 - val_accuracy: 0

In [14]:
y_probs = model.predict(X_test).ravel()
y_pred = (y_probs > 0.5).astype(int)

# 8. Resultados
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step
Confusion Matrix:
 [[  76 1252]
 [  62 8610]]

Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.06      0.10      1328
           1       0.87      0.99      0.93      8672

    accuracy                           0.87     10000
   macro avg       0.71      0.53      0.52     10000
weighted avg       0.83      0.87      0.82     10000

F1 Score: 0.9291032696665588


## Tunner 1

In [15]:
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = {0: weights[0], 1: weights[1]}

print(class_weight)

{0: np.float64(3.930817610062893), 1: np.float64(0.5728689275893676)}


In [18]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = {0: weights[0], 1: weights[1]}


def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(hp.Int('units1', 32, 128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout1', 0.0, 0.5, step=0.1)))
    model.add(Dense(hp.Int('units2', 16, 64, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=[Precision()]
    )
    return model


from keras_tuner.engine import tuner as kt_engine

class MyTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['class_weight'] = class_weight
        return super().run_trial(trial, *args, **kwargs)

tuner = MyTuner(
    build_model,
    objective='val_precision',  
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory='tuning_dir',
    project_name='maximize_precision'
)


tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=1)


best_model = tuner.get_best_models(1)[0]


y_pred = (best_model.predict(X_val).ravel() > 0.5).astype(int)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

Trial 10 Complete [00h 00m 07s]
val_precision: 0.9446634650230408

Best val_precision So Far: 0.94524747133255
Total elapsed time: 00h 01m 05s


/Users/martinagarciagonzalez/Library/Python/3.12/lib/python/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step
[[ 953  375]
 [2198 6474]]
              precision    recall  f1-score   support

           0       0.30      0.72      0.43      1328
           1       0.95      0.75      0.83      8672

    accuracy                           0.74     10000
   macro avg       0.62      0.73      0.63     10000
weighted avg       0.86      0.74      0.78     10000



In [19]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

{'units1': 32, 'dropout1': 0.2, 'units2': 32, 'lr': 0.005756624545738224}

In [22]:
model = build_model(best_hp)
model.fit(X_train, y_train, 
          epochs=10, 
          validation_data=(X_val, y_val), 
          class_weight=class_weight)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step - loss: 0.6162 - precision_2: 0.9348 - val_loss: 0.5677 - val_precision_2: 0.9432
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 441us/step - loss: 0.5754 - precision_2: 0.9470 - val_loss: 0.5885 - val_precision_2: 0.9442
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 425us/step - loss: 0.5744 - precision_2: 0.9473 - val_loss: 0.5909 - val_precision_2: 0.9447
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 421us/step - loss: 0.5772 - precision_2: 0.9469 - val_loss: 0.5485 - val_precision_2: 0.9446
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 430us/step - loss: 0.5737 - precision_2: 0.9469 - val_loss: 0.6404 - val_precision_2: 0.9453
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 417us/step - loss: 0.5682 - precision_2: 0.9488 - val_loss: 0.5315 - val_precision_2: 0.9438
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 419us/step - loss: 0.5637 - precision_2: 0.9488 - val_loss: 0.6183 - val_precision_2: 0.9443
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━

In [23]:
# Predecir sobre el conjunto de validación
y_probs = model.predict(X_val).ravel()
y_pred = (y_probs > 0.5).astype(int)

# Métricas de evaluación
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

print(f"\nF1 Score: {f1_score(y_val, y_pred):.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step
Confusion Matrix:
[[ 944  384]
 [2163 6509]]

Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.71      0.43      1328
           1       0.94      0.75      0.84      8672

    accuracy                           0.75     10000
   macro avg       0.62      0.73      0.63     10000
weighted avg       0.86      0.75      0.78     10000


F1 Score: 0.8364


In [32]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight = {0: weights[0], 1: weights[1]}


def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(hp.Int('units1', 32, 128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout1', 0.0, 0.5, step=0.1)))
    model.add(Dense(hp.Int('units2', 16, 64, step=16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
    optimizer=Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
    loss='binary_crossentropy',
    metrics=[Precision(name='precision'), Recall(name='recall')])
    
    return model


from keras_tuner.engine import tuner as kt_engine

class MyTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['class_weight'] = class_weight
        return super().run_trial(trial, *args, **kwargs)

tuner = MyTuner(
    build_model,
    objective='val_recall',  
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory='tuning_dir',
    project_name='maximize_recall'
)

tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=32, verbose=1)


best_model = tuner.get_best_models(1)[0]


y_pred = (best_model.predict(X_val).ravel() > 0.5).astype(int)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

Trial 10 Complete [00h 00m 22s]
val_recall: 0.7606088519096375

Best val_recall So Far: 0.767988920211792
Total elapsed time: 00h 03m 35s


/Users/martinagarciagonzalez/Library/Python/3.12/lib/python/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step
[[ 909  419]
 [2012 6660]]
              precision    recall  f1-score   support

           0       0.31      0.68      0.43      1328
           1       0.94      0.77      0.85      8672

    accuracy                           0.76     10000
   macro avg       0.63      0.73      0.64     10000
weighted avg       0.86      0.76      0.79     10000

